In [1]:
import pandas as pd
from pulp import *

### Read the given nutrition dataset into a Pandas DataFrame object
Note we are reading only the first 64 rows with `nrows=64` argument because we just want to read all the nutrients informtion and not the maximum/minimum bounds in the dataset. We will enter those bounds in the optimization problem separately.

In [2]:
df = pd.read_excel("Data/diet.xls",nrows=64)

### Show first 5 rows of the dataset

In [3]:
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


### Create the `PuLP` problem variable. Since it is a cost minimization problem, we need to use `LpMinimize`

In [4]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Simple Diet Problem",LpMinimize)

/home/andreas/anaconda3/envs/py38/lib/python3.8/site-packages/pulp/pulp.py:1190: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Create a list of food items from the dataset

In [5]:
# Creates a list of the Ingredients
food_items = list(df['Foods'])

In [6]:
print("So, the food items to consdier, are\n"+"-"*100)
for f in food_items:
    print(f,end=', ')

So, the food items to consdier, are
----------------------------------------------------------------------------------------------------
Frozen Broccoli, Carrots,Raw, Celery, Raw, Frozen Corn, Lettuce,Iceberg,Raw, Peppers, Sweet, Raw, Potatoes, Baked, Tofu, Roasted Chicken, Spaghetti W/ Sauce, Tomato,Red,Ripe,Raw, Apple,Raw,W/Skin, Banana, Grapes, Kiwifruit,Raw,Fresh, Oranges, Bagels, Wheat Bread, White Bread, Oatmeal Cookies, Apple Pie, Chocolate Chip Cookies, Butter,Regular, Cheddar Cheese, 3.3% Fat,Whole Milk, 2% Lowfat Milk, Skim Milk, Poached Eggs, Scrambled Eggs, Bologna,Turkey, Frankfurter, Beef, Ham,Sliced,Extralean, Kielbasa,Prk, Cap'N Crunch, Cheerios, Corn Flks, Kellogg'S, Raisin Brn, Kellg'S, Rice Krispies, Special K, Oatmeal, Malt-O-Meal,Choc, Pizza W/Pepperoni, Taco, Hamburger W/Toppings, Hotdog, Plain, Couscous, White Rice, Macaroni,Ckd, Peanut Butter, Pork, Sardines in Oil, White Tuna in Water, Popcorn,Air-Popped, Potato Chips,Bbqflvr, Pretzels, Tortilla Chip, Chicknood

### Create a dictinary of costs for all food items

In [7]:
costs = dict(zip(food_items,df['Price/ Serving']))

In [8]:
costs

{'Frozen Broccoli': 0.16,
 'Carrots,Raw': 0.07,
 'Celery, Raw': 0.04,
 'Frozen Corn': 0.18,
 'Lettuce,Iceberg,Raw': 0.02,
 'Peppers, Sweet, Raw': 0.53,
 'Potatoes, Baked': 0.06,
 'Tofu': 0.31,
 'Roasted Chicken': 0.84,
 'Spaghetti W/ Sauce': 0.78,
 'Tomato,Red,Ripe,Raw': 0.27,
 'Apple,Raw,W/Skin': 0.24,
 'Banana': 0.15,
 'Grapes': 0.32,
 'Kiwifruit,Raw,Fresh': 0.49,
 'Oranges': 0.15,
 'Bagels': 0.16,
 'Wheat Bread': 0.05,
 'White Bread': 0.06,
 'Oatmeal Cookies': 0.09,
 'Apple Pie': 0.16,
 'Chocolate Chip Cookies': 0.03,
 'Butter,Regular': 0.05,
 'Cheddar Cheese': 0.25,
 '3.3% Fat,Whole Milk': 0.16,
 '2% Lowfat Milk': 0.23,
 'Skim Milk': 0.13,
 'Poached Eggs': 0.08,
 'Scrambled Eggs': 0.11,
 'Bologna,Turkey': 0.15,
 'Frankfurter, Beef': 0.27,
 'Ham,Sliced,Extralean': 0.33,
 'Kielbasa,Prk': 0.15,
 "Cap'N Crunch": 0.31,
 'Cheerios': 0.28,
 "Corn Flks, Kellogg'S": 0.28,
 "Raisin Brn, Kellg'S": 0.34,
 'Rice Krispies': 0.32,
 'Special K': 0.38,
 'Oatmeal': 0.82,
 'Malt-O-Meal,Choc': 0.52,
 

### Create a dictionary of calories for all food items

In [9]:
calories = dict(zip(food_items,df['Calories']))

### Create a dictionary of cholesterol for all food items

In [10]:
cholesterol = dict(zip(food_items,df['Cholesterol mg']))

### Create a dictionary of total fat for all food items

In [11]:
fat = dict(zip(food_items,df['Total_Fat g']))

### Create a dictionary of sodium for all food items

In [12]:
sodium = dict(zip(food_items,df['Sodium mg']))

### Create a dictionary of carbohydrates for all food items

In [13]:
carbs = dict(zip(food_items,df['Carbohydrates g']))

### Create a dictionary of dietary fiber for all food items

In [14]:
fiber = dict(zip(food_items,df['Dietary_Fiber g']))

### Create a dictionary of protein for all food items

In [15]:
protein = dict(zip(food_items,df['Protein g']))

### Create a dictionary of vitamin A for all food items

In [16]:
vit_A = dict(zip(food_items,df['Vit_A IU']))

### Create a dictionary of vitamin C for all food items

In [17]:
vit_C = dict(zip(food_items,df['Vit_C IU']))

### Create a dictionary of calcium for all food items

In [18]:
calcium = dict(zip(food_items,df['Calcium mg']))

### Create a dictionary of iron for all food items

In [19]:
iron = dict(zip(food_items,df['Iron mg']))

### Create a dictionary of food portion with lower bound 0 - these are the main optimization variables

In [20]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_vars = LpVariable.dicts("Portion",food_items,0,cat='Continuous')

### Create another set of variables for each food, with integer 0 or 1. These are indicator variables

In [21]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_chosen = LpVariable.dicts("Chosen",food_items,0,1,cat='Integer')

### Adding the objective function to the problem

In [22]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

### Adding the calorie constraints to the problem

In [23]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 1500.0, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 2500.0, "CalorieMaximum"

### Adding other nutrient constraints to the problem one by one...

In [24]:
# Cholesterol
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) >= 30.0, "CholesterolMinimum"
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) <= 240.0, "CholesterolMaximum"

# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 70.0, "FatMaximum"

# Sodium
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) >= 800.0, "SodiumMinimum"
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) <= 2000.0, "SodiumMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 450.0, "CarbsMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 125.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 250.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 60.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 100.0, "ProteinMaximum"

# Vitamin A
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) >= 1000.0, "VitaminAMinimum"
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) <= 10000.0, "VitaminAMaximum"

# Vitamin C
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) >= 400.0, "VitaminCMinimum"
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) <= 5000.0, "VitaminCMaximum"

# Calcium
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) >= 700.0, "CalciumMinimum"
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) <= 1500.0, "CalciumMaximum"

# Iron
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) >= 10.0, "IronMinimum"
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) <= 40.0, "IronMaximum"

### Adding constraint linking `food_vars` and `food_chosen`

In [25]:
for f in food_items:
    prob += food_vars[f]>= food_chosen[f]*0.1
    prob += food_vars[f]<= food_chosen[f]*1e5

### Adding constraint of celery and frozen broccoli

In [26]:
prob += food_chosen['Frozen Broccoli']+food_chosen['Celery, Raw']<=1

### Adding constraint of at least 3 types of meat/poultry/fish/eggs in every diet

In [27]:
protein_choices = ['Beanbacn Soup,W/Watr','Bologna,Turkey','Frankfurter, Beef','Ham,Sliced,Extralean',
                  'Hamburger W/Toppings','Hotdog, Plain','Kielbasa,Prk','Neweng Clamchwd','Pizza W/Pepperoni',
                  'Poached Eggs','Pork','Roasted Chicken','Sardines in Oil','Scrambled Eggs','Vegetbeef Soup',
                   'White Tuna in Water']

In [28]:
prob += lpSum([food_chosen[p] for p in protein_choices]) >= 3.0

### Writing problem data to a `.lp` file

In [29]:
# The problem data is written to an .lp file
prob.writeLP("SimpleDietProblem.lp")

[Chosen_2%_Lowfat_Milk,
 Chosen_3.3%_Fat,Whole_Milk,
 Chosen_Apple,Raw,W_Skin,
 Chosen_Apple_Pie,
 Chosen_Bagels,
 Chosen_Banana,
 Chosen_Beanbacn_Soup,W_Watr,
 Chosen_Bologna,Turkey,
 Chosen_Butter,Regular,
 Chosen_Cap'N_Crunch,
 Chosen_Carrots,Raw,
 Chosen_Celery,_Raw,
 Chosen_Cheddar_Cheese,
 Chosen_Cheerios,
 Chosen_Chicknoodl_Soup,
 Chosen_Chocolate_Chip_Cookies,
 Chosen_Corn_Flks,_Kellogg'S,
 Chosen_Couscous,
 Chosen_Crm_Mshrm_Soup,W_Mlk,
 Chosen_Frankfurter,_Beef,
 Chosen_Frozen_Broccoli,
 Chosen_Frozen_Corn,
 Chosen_Grapes,
 Chosen_Ham,Sliced,Extralean,
 Chosen_Hamburger_W_Toppings,
 Chosen_Hotdog,_Plain,
 Chosen_Kielbasa,Prk,
 Chosen_Kiwifruit,Raw,Fresh,
 Chosen_Lettuce,Iceberg,Raw,
 Chosen_Macaroni,Ckd,
 Chosen_Malt_O_Meal,Choc,
 Chosen_New_E_Clamchwd,W_Mlk,
 Chosen_Neweng_Clamchwd,
 Chosen_Oatmeal,
 Chosen_Oatmeal_Cookies,
 Chosen_Oranges,
 Chosen_Peanut_Butter,
 Chosen_Peppers,_Sweet,_Raw,
 Chosen_Pizza_W_Pepperoni,
 Chosen_Poached_Eggs,
 Chosen_Popcorn,Air_Popped,
 Chosen_

### Run the solver

In [30]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

### Print the problem solution status `'optimal'`, `'infeasible'`, `'unbounded'` etc...

In [31]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


### Scan through the problem variables and print out only if the variable quanity is positive i.e. it is included in the optimal solution

In [32]:
print("The optimal (least cost) balanced diet with additional constraints \
(e.g. at least 3 types of animal protein sources, consists of\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0 and v.name[0]=='P':
        print(v.name, "=", v.varValue)

The optimal (least cost) balanced diet with additional constraints (e.g. at least 3 types of animal protein sources, consists of
--------------------------------------------------------------------------------------------------------------
Portion_Celery,_Raw = 42.399358
Portion_Kielbasa,Prk = 0.1
Portion_Lettuce,Iceberg,Raw = 82.802586
Portion_Oranges = 3.0771841
Portion_Peanut_Butter = 1.9429716
Portion_Poached_Eggs = 0.1
Portion_Popcorn,Air_Popped = 13.223294
Portion_Scrambled_Eggs = 0.1


### Print the optimal diet cost

In [33]:
print("The total cost of this balanced diet is: ${}".format(round(value(prob.objective),2)))

The total cost of this balanced diet is: $4.51
